In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import math
import itertools
from itertools import permutations as p
import functools
import re
import random
import collections
from collections import Counter

In [2]:
def yielder(x):
    for y in x:
        if isinstance(y, tuple):
            yield from yielder(y)
        else:
            yield y

In [3]:
def generate_matrix(alphabet): #we are considering 2 by 2 sub matrices
    row_lst = list(itertools.product(alphabet, alphabet))
    matrix_lst = []
    for i in range(len(row_lst)):
        for j in range(len(row_lst)):
            matrix = [row_lst[i],row_lst[j]]
            matrix_lst.append(matrix)
    lst = [[*yielder(e)] for e in matrix_lst]
    return lst

In [4]:
def pieceable(m1,m2):
    #identify if two matrices can be put next to each other
    lst_1 = m1.strip('][').split(', ')
    lst_2 = m2.strip('][').split(', ')
    orientation = []
    indicator = 0
    if (lst_1[0]==lst_2[2] and lst_1[1]==lst_2[3]):
        orientation.append(["up", [lst_1[0], lst_1[1]]])
        indicator = 1
    """if (lst_1[0]==lst_2[1] and lst_1[2]==lst_2[3]):
        orientation.append(["left", [lst_1[0], lst_1[2]]])
        indicator = 1
    if (lst_1[2]==lst_2[0] and lst_1[3]==lst_2[1]):
        orientation.append(["down", [lst_1[2], lst_1[3]]])
        indicator = 1"""
    if (lst_1[1]==lst_2[0] and lst_1[3]==lst_2[2]):
        orientation.append(["right", [lst_1[1], lst_1[3]]])
        indicator = 1
    if indicator == 1:
        return [True, orientation]
    else:
        return [False, orientation]

In [5]:
#Now we want to write a function that edtablish direct edges between sub matrices (vertices)
def build_edge(matrix_lst):
    edge_lst = []
    for i in range(len(matrix_lst)):
        for j in range(i+1, len(matrix_lst)):
            if pieceable(str(matrix_lst[i]), str(matrix_lst[j]))[0]:
                edge_lst.append([matrix_lst[i], pieceable(str(matrix_lst[i]), str(matrix_lst[j]))[1], matrix_lst[j]])        
    return edge_lst

In [6]:
matrix_lst = generate_matrix([1,2,3])
build_edge(matrix_lst)

[[[1, 1, 1, 1], [['right', ['1', '1']]], [1, 1, 1, 2]],
 [[1, 1, 1, 1], [['right', ['1', '1']]], [1, 1, 1, 3]],
 [[1, 1, 1, 1], [['up', ['1', '1']], ['right', ['1', '1']]], [1, 2, 1, 1]],
 [[1, 1, 1, 1], [['right', ['1', '1']]], [1, 2, 1, 2]],
 [[1, 1, 1, 1], [['right', ['1', '1']]], [1, 2, 1, 3]],
 [[1, 1, 1, 1], [['up', ['1', '1']], ['right', ['1', '1']]], [1, 3, 1, 1]],
 [[1, 1, 1, 1], [['right', ['1', '1']]], [1, 3, 1, 2]],
 [[1, 1, 1, 1], [['right', ['1', '1']]], [1, 3, 1, 3]],
 [[1, 1, 1, 1], [['up', ['1', '1']]], [2, 1, 1, 1]],
 [[1, 1, 1, 1], [['up', ['1', '1']]], [2, 2, 1, 1]],
 [[1, 1, 1, 1], [['up', ['1', '1']]], [2, 3, 1, 1]],
 [[1, 1, 1, 1], [['up', ['1', '1']]], [3, 1, 1, 1]],
 [[1, 1, 1, 1], [['up', ['1', '1']]], [3, 2, 1, 1]],
 [[1, 1, 1, 1], [['up', ['1', '1']]], [3, 3, 1, 1]],
 [[1, 1, 1, 2], [['right', ['1', '2']]], [1, 1, 2, 1]],
 [[1, 1, 1, 2], [['right', ['1', '2']]], [1, 1, 2, 2]],
 [[1, 1, 1, 2], [['right', ['1', '2']]], [1, 1, 2, 3]],
 [[1, 1, 1, 2], [['up', ['

In [7]:
def eulerian_path(alphabet):
    matrix_lst = generate_matrix(alphabet)
    edge_lst = build_edge(matrix_lst)
    new_lst = []
    for item in edge_lst:
        start_vertex = str(item[0])
        end_vertex = str(item[2])
        new_lst.append((start_vertex, end_vertex))
    #random.shuffle(new_lst)
    G = nx.DiGraph()
    G.add_edges_from(new_lst)
    return [u for u, v in nx.eulerian_circuit(G)]

In [8]:
eulerian_path([1,2,3])

NetworkXError: G is not Eulerian.

In [9]:
def generate_detorus(alphabet):
    vertex_lst = eulerian_path(alphabet)
    ori_lst = []
    for i in range(len(vertex_lst)):
        if i < len(vertex_lst) - 1:
            orientation = pieceable(vertex_lst[i], vertex_lst[i+1])[1]
        else:
            orientation = pieceable(vertex_lst[i], vertex_lst[0])[1]
        ori_lst.append(orientation)
    return (vertex_lst, ori_lst)

In [10]:
#Now we want to confirm our made-up formula, if it is confirmed, then we try to piece tgt the 9 by 9 matrix
def confirm_square(edges):
    orientation_lst = []
    bridge_lst = []
    for i in range(len(edges)):
        if len(edges[i]) == 1:
            orientation_lst.append(edges[i][0][0])
            bridge_lst.append(edges[i][0][1])
    summary = Counter(orientation_lst)
    return summary == Counter({'down': 504, 'right': 504, 'left': 504, 'up': 504})

In [11]:
def simplify(edges_lst):
    #for those that have multiple orientation, we will pick the first one in the list
    edges = edges_lst.copy()
    counter = 0
    for i in range(len(edges)):
        if len(edges[i]) != 1: #we pick an orientation that is different from what we used previously
            for j in range(len(edges[i])):
                if i != 0:
                    if edges[i-1][0][0] == "up": 
                        if edges[i][j][0] != "down":
                            edges[i] = [edges[i][j]]
                            break
                    elif edges[i-1][0][0] == "down":
                        if edges[i][j][0] != "up":
                            edges[i] = [edges[i][j]]
                            break
                    elif edges[i-1][0][0] == "left":
                        if edges[i][j][0] != "right":
                            edges[i] = [edges[i][j]]
                            break
                    elif edges[i-1][0][0] == "right":
                        if edges[i][j][0] != "left":
                            edges[i] = [edges[i][j]]
                            break
                else:
                    if edges[80][0][0] == "up": 
                        if edges[i][j][0] != "down":
                            edges[i] = [edges[i][j]]
                            break
                    elif edges[80][0][0] == "down":
                        if edges[i][j][0] != "up":
                            edges[i] = [edges[i][j]]
                            break
                    elif edges[80][0][0] == "left":
                        if edges[i][j][0] != "right":
                            edges[i] = [edges[i][j]]
                            break
                    elif edges[80][0][0] == "right":
                        if edges[i][j][0] != "left":
                            edges[i] = [edges[i][j]]
                            break
    return edges

In [12]:
def merge(matrix1, matrix2, orientation, grid):
    matrix1_pos = grid.index(matrix1)
    if orientation == "right":
        grid[(matrix1_pos+1)%81] = matrix2
    elif orientation == "left":
        grid[(matrix1_pos-1)%81] = matrix2
    elif orientation == "up":
        grid[(matrix1_pos-9)%81] = matrix2
    elif orientation == "down":
        grid[(matrix1_pos+9)%81] = matrix2
    return grid

In [13]:
def piece_tgt(vertices, simplified_edges):
    grid = [
           [],[],[],[],[],[],[],[],[],
           [],[],[],[],[],[],[],[],[],
           [],[],[],[],[],[],[],[],[],
           [],[],[],[],[],[],[],[],[],
           [],[],[],[],[],[],[],[],[],
           [],[],[],[],[],[],[],[],[],
           [],[],[],[],[],[],[],[],[],
           [],[],[],[],[],[],[],[],[],
           [],[],[],[],[],[],[],[],[],
           ]
    grid[40] = vertices[0]
    for i in range(len(vertices)-1):
        grid = merge(vertices[i], vertices[i+1], simplified_edges[i][0][0], grid)
        print(grid)
        print("----------------------------------")
    #return grid

In [14]:
 def compare_matrix(m1, m2):
        comparison = m1 == m2
        equal_arrays = comparison.all()
        return equal_arrays

In [15]:
def is_a_torus(matrix2):
    sub_matrix_list = []
    for i in range (9):
        for j in range(9):
            if i == 8 and j != 8:
                sub_matrix_list.append(matrix2[np.ix_([i, 0],[j, j+1])])
            elif j == 8 and i != 8:
                sub_matrix_list.append(matrix2[np.ix_([i, i+1],[j, 0])])
            elif i == 8 and j == 8:
                sub_matrix_list.append(matrix2[np.ix_([i, 0],[j, 0])])
            else:
                sub_matrix_list.append(matrix2[np.ix_([i, i+1],[j, j+1])]) #first list are rows, second list are columns
    length = (len(sub_matrix_list))
    for i in range(length):
        for j in range(i+1, length):
            if compare_matrix(sub_matrix_list[i], sub_matrix_list[j]): #return true if two matrix are the same
                return False
    return True

In [16]:
def convert(final_grid):
    #we need to convert the grids into actual numpy matrix
    list1 = []
    list2 = []
    list3 = []
    list4 = []
    list5 = []
    list6 = []
    list7 = []
    list8 = []
    list9 = []
    for i in range(len(final_grid)):
        not_converted = final_grid[i]
        converted = to_int(not_converted)
        if 0 <= i <= 8:
            list1.append(converted[0])
            list2.append(converted[2])
        elif 18 <= i <= 26:
            list3.append(converted[0])
            list4.append(converted[2])
        elif 36 <= i <= 44:
            list5.append(converted[0])
            list6.append(converted[2])
        elif 54 <= i <= 62:
            list7.append(converted[0])
            list8.append(converted[2])
        elif 63 <= i <= 71:
            list9.append(converted[2])
    matrix2 = np.matrix([list1,list2,list3,list4,list5,list6,list7,list8,list9])
    return matrix2

In [17]:
def to_int(lst):
    #turn '[1, 2, 1, 1]' to [1,2,1,1]
    new_lst = lst.strip('][').split(', ')
    A = [int(x) for x in new_lst]
    return A

In [18]:
def generate(how_many):
    all_torus = []
    count = 0
    for i in range(how_many):
        generater = generate_detorus([0,1,2])
        edges = generater[1]
        vertices = generater[0]
        simplified_edges = simplify(edges)
        final_grid = piece_tgt(vertices, simplified_edges)
        if [] not in final_grid:
            de_bruijn_torus = convert(final_grid)
            if is_a_torus(de_bruijn_torus):
                all_torus.append(de_bruijn_torus)
                print (de_bruijn_torus)
    return all_torus

In [19]:
generated = generate_detorus([0,1,2])

NetworkXError: G is not Eulerian.

In [20]:
vertices = generated[0]
edges = generated[1]
edges

NameError: name 'generated' is not defined

In [21]:
list1 = [0,0,0,2,2,0,2,2,0]
list2 = [0,0,1,2,1,1,1,2,1]
list3 = [1,1,2,0,1,2,1,0,2]
list4 = [2,2,2,2,0,2,0,2,2]
list5 = [2,2,0,1,0,0,0,1,0]
list6 = [0,0,2,1,1,0,0,1,2]
list7 = [2,2,1,0,2,1,2,0,1]
list8 = [1,1,1,0,2,1,2,0,1]
list9 = [1,1,0,1,0,0,0,1,0]

matrix2 = np.matrix([list1,list2,list3,list4,list5,list6,list7,list8,list9])
is_a_torus(matrix2)

#So this turns out to work: 
#1. find a de Bruijn sequence of size 2 with alphabet [0,1,2]
#3 Now pick one sequence, write it as a column, in the above example we picked 001220211
#4 Now we can generate column 2 to 9 by shifting the sequence upwards by 0,1,2,3,4,5,6,7


False

In [22]:
def sequence_to_torus(de_bruijn_sequence):
    list1 = []
    list2 = []
    list3 = []
    list4 = []
    list5 = []
    list6 = []
    list7 = []
    list8 = []
    list9 = []
    collection = [list1, list2, list3, list4, list5, list6, list7, list8, list9]
    for i in range(9):
        collection[i].append(int(de_bruijn_sequence[i]))
        collection[i].append(int(de_bruijn_sequence[i]))
        collection[i].append(int(de_bruijn_sequence[(i+1)%9]))
        collection[i].append(int(de_bruijn_sequence[(i+3)%9]))
        collection[i].append(int(de_bruijn_sequence[(i+6)%9]))
        collection[i].append(int(de_bruijn_sequence[(i+10)%9]))
        collection[i].append(int(de_bruijn_sequence[(i+15)%9]))
        collection[i].append(int(de_bruijn_sequence[(i+21)%9]))
        collection[i].append(int(de_bruijn_sequence[(i+28)%9]))
    matrix = np.matrix(collection)
    return matrix

In [25]:
torus = sequence_to_torus('001220211')
torus

matrix([[0, 0, 0, 2, 2, 0, 2, 2, 0],
        [0, 0, 1, 2, 1, 1, 1, 2, 1],
        [1, 1, 2, 0, 1, 2, 1, 0, 2],
        [2, 2, 2, 2, 0, 2, 0, 2, 2],
        [2, 2, 0, 1, 0, 0, 0, 1, 0],
        [0, 0, 2, 1, 1, 2, 1, 1, 2],
        [2, 2, 1, 0, 2, 1, 2, 0, 1],
        [1, 1, 1, 0, 2, 1, 2, 0, 1],
        [1, 1, 0, 1, 0, 0, 0, 1, 0]])

In [ ]:
sequence_lst = ['022112010','200110212', '011221020',
'211001202',
'122002101',
'100220121']

matrix_lst = []
for sequence in sequence_lst:
    matrix_lst.append(sequence_to_torus(sequence))

In [26]:
def orthogonal(m1, m2):
    for i in range(81):
        if m1.item(i) == m2.item(i):
            return False
    return True

In [ ]:
example = matrix_lst[0]
example

In [ ]:
example[8:9, 8:9]

In [ ]:
top_row = np.c_[example[8:9, 8:9], example[8:9, 0:1]]
bottom_row = np.c_[example[0:1, 8:9], example[0:1, 0:1]]
np.r_[top_row, bottom_row]
example[8:9, 1:3]

In [27]:
def print_latex(matrix):
    #We use the same table as our article to translate 0,1,2. In the context of LaTeX code, 
    # number: 0 = 3, 1 = 1, 2 = 2
    # color: 0 = 3, 1 = 1, 2 = 2
    # filling: 0 = 2, 1 = 3, 2 = 1
    # shape: 0 = 1, 1 = 2, 2 = 3
    # number, color, filling, shape becomes number, filling, color, shape in LaTeX
    sub_lst = []
    string = ''
    translator = {'number': {0:3, 1:1, 2:2}, 'color':{0:3, 1:1, 2:2}, 'filling':{0:2, 1:3, 2:1}, 'shape':{0:1, 1:2, 2:3}}
    for i in range(9):
        for j in range(9):
            if j != 8 and i != 8:
                sub = matrix[i: i+2, j: j+2]
            elif j == 8 and i != 8:
                sub = np.c_[matrix[i:i+2, 8:9], matrix[i:i+2, 0:1]]
            elif j != 8 and i == 8:
                sub = np.r_[matrix[8:9, j:j+2], matrix[0:1, j:j+2]]
            else:
                top_row = np.c_[matrix[8:9, 8:9], matrix[8:9, 0:1]]
                bottom_row = np.c_[matrix[0:1, 8:9], matrix[0:1, 0:1]]
                sub = np.r_[top_row, bottom_row]
            sub_lst.append(sub)
    for i in range(len(sub_lst)):
        lst = sub_lst[i].tolist()
        number = lst[0][0]
        color = lst[0][1]
        filling = lst[1][0]
        shape = lst[1][1]
        latex_number = translator['number'][number]
        latex_color = translator['color'][color]
        latex_filling = translator['filling'][filling]
        latex_shape = translator['shape'][shape]
        if i%9 == 8:
            string += '{'+'slashsetcard'+'{'+str(latex_number)+'}'+'{'+str(latex_filling)+'}'+'{'+str(latex_color)+'}'+'{'+str(latex_shape)+'}' +'{0.2}'+'}'+'double'
        else:
            string += '{'+'slashsetcard'+'{'+str(latex_number)+'}'+'{'+str(latex_filling)+'}'+'{'+str(latex_color)+'}'+'{'+str(latex_shape)+'}'+'{0.2}'+'}'+'&'
    print (string) #copy and paste the string to LaTeX, replace "slash" with "/", and "double" with "//" 

In [28]:
pip install array-to-latex

  Using cached array_to_latex-0.82-py3-none-any.whl (5.3 kB)
Processing ./.cache/pip/wheels/de/bc/35/fe31f0f559f65513fb90922017715133ed34d148f8eff157f4/clipboard-0.0.4-py3-none-any.whl
Processing ./.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122/pyperclip-1.8.2-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [29]:
import array_to_latex as a2l

In [30]:
"""for matrix in matrix_lst:
    latex_code = a2l.to_ltx(matrix, frmt = '{:0.0f}', arraytype = 'matrix')
    print_latex(matrix)"""

"for matrix in matrix_lst:\n    latex_code = a2l.to_ltx(matrix, frmt = '{:0.0f}', arraytype = 'matrix')\n    print_latex(matrix)"

In [31]:
latex_code = a2l.to_ltx(torus, frmt = '{:0.0f}', arraytype = 'matrix')
print_latex(torus)

\begin{matrix}
  0  &  0  &  0  &  2  &  2  &  0  &  2  &  2  &  0 \\
  0  &  0  &  1  &  2  &  1  &  1  &  1  &  2  &  1 \\
  1  &  1  &  2  &  0  &  1  &  2  &  1  &  0  &  2 \\
  2  &  2  &  2  &  2  &  0  &  2  &  0  &  2  &  2 \\
  2  &  2  &  0  &  1  &  0  &  0  &  0  &  1  &  0 \\
  0  &  0  &  2  &  1  &  1  &  2  &  1  &  1  &  2 \\
  2  &  2  &  1  &  0  &  2  &  1  &  2  &  0  &  1 \\
  1  &  1  &  1  &  0  &  2  &  1  &  2  &  0  &  1 \\
  1  &  1  &  0  &  1  &  0  &  0  &  0  &  1  &  0 
\end{matrix}
{slashsetcard{3}{2}{3}{1}{0.2}}&{slashsetcard{3}{2}{3}{2}{0.2}}&{slashsetcard{3}{3}{2}{3}{0.2}}&{slashsetcard{2}{1}{2}{2}{0.2}}&{slashsetcard{2}{3}{3}{2}{0.2}}&{slashsetcard{3}{3}{2}{2}{0.2}}&{slashsetcard{2}{3}{2}{3}{0.2}}&{slashsetcard{2}{1}{3}{2}{0.2}}&{slashsetcard{3}{3}{3}{1}{0.2}}double{slashsetcard{3}{3}{3}{2}{0.2}}&{slashsetcard{3}{3}{1}{3}{0.2}}&{slashsetcard{1}{1}{2}{1}{0.2}}&{slashsetcard{2}{2}{1}{2}{0.2}}&{slashsetcard{1}{3}{1}{3}{0.2}}&{slashsetcard{1}{1}{1}{2}{

In [ ]:
'{' + 'slashsetcard' +'{'+ str(0) +'}'+'{'+str(3)+'}'+'{'+str(2)+'}'+'{'+str(2)+'}'+ '}' + '\\'

In [ ]:
print (r'\setcard')